In [1]:
# Import Libraries
import sqlite3
import pandas as pd

# Create database
db = sqlite3.connect("transactions.db")


In [2]:
# Connect to database
con = sqlite3.connect("/Users/DJ/E-Commerce-Recommendations/transactions.db")

In [3]:
# Load txt files into pandas and separate columns
adhesiveSealantTrans = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/Transactions with A&S.txt",sep='\t', header=0)
allTransaction = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/All Transations - 2 Weeks.txt", sep='\t', header=0)

In [4]:
# Check dataframes
adhesiveSealantTrans.head(5)

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


In [5]:
allTransaction.head(5)

,order_number,l1,l2,l3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603


In [7]:
# Move dataframes to sql tables in transactions.db
adhesiveSealantTrans.to_sql("aNs_transactions", con, if_exists="replace", index=False)
allTransaction.to_sql("all_trans", con, if_exists="replace", index=False)

In [24]:
pd.read_sql_query("SELECT 'sku', COUNT('sku') AS 'occurrence' FROM 'aNs_transactions' GROUP BY 'sku' ORDER BY COUNT('occurence') DESC;", con)

,'sku',occurrence
0,sku,3387279
